In [1]:
# Import libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmmml import MLPotential
import sys

# Setup
pdb = PDBFile("aaa.pdb")
pdb.topology.setPeriodicBoxVectors(None)
potential = MLPotential('ani2x')
system = potential.createSystem(pdb.topology)

/home/mbowley/miniconda3/envs/ani/lib/python3.9/site-packages/torchani/__init__.py:55: UserWarning: Dependency not satisfied, torchani.ase will not be available
  warnings.warn("Dependency not satisfied, torchani.ase will not be available")
/home/mbowley/miniconda3/envs/ani/lib/python3.9/site-packages/torch/functional.py:1069: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1640869844479/work/aten/src/ATen/native/TensorShape.cpp:2156.)
  return _VF.cartesian_prod(tensors)  # type: ignore[attr-defined]


/home/mbowley/miniconda3/envs/ani/lib/python3.9/site-packages/torchani/resources/


# Works fine on CPU (slow)

In [2]:
integrator = LangevinIntegrator(
    300 * kelvin, 
    1 / picosecond, 
    1.0 * femtosecond,
)
simulation = Simulation(
    pdb.topology,
    system,
    integrator,
    Platform.getPlatformByName("CPU"),
)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
print("done")

[W BinaryOps.cpp:595] Warning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (function operator())


done


# Breaks with CUDA

In [2]:
integrator = LangevinIntegrator(
    300 * kelvin, 
    1 / picosecond, 
    1.0 * femtosecond,
)
simulation = Simulation(
    pdb.topology,
    system,
    integrator,
    Platform.getPlatformByName("CUDA"),
)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.step(100)
print("done")

[W LegacyTypeDispatch.h:79] Warning: AutoNonVariableTypeMode is deprecated and will be removed in 1.10 release. For kernel implementations please use AutoDispatchBelowADInplaceOrView instead, If you are looking for a user facing API to enable running your inference-only workload, please use c10::InferenceMode. Using AutoDispatchBelowADInplaceOrView in user code is under risk of producing silent wrong result in some edge cases. See Note [AutoDispatchBelowAutograd] for more details. (function operator())


# Troubleshooting

In [ ]:
!conda list | grep -E "torch|openmm"

openmm                    7.7.0            py39h792354b_0    conda-forge
openmm-torch              0.5             cuda112py39hb628e3f_0    conda-forge
openmmml                  1.0                      pypi_0    pypi
pytorch                   1.10.0          cuda112py39h3ad47f5_1    conda-forge
pytorch-gpu               1.10.0          cuda112py39h0bbbad9_1    conda-forge
torchani                  2.2.3.dev2+g3dfbaf4          pypi_0    pypi


In [ ]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
0
4
NVIDIA GeForce GTX 1080
